# Segmenting and Clustering Neighborhoods in Toronto - General Part :

<span style="font-size:18px"> Explore and cluster the neighborhoods in Toronto. This part will work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data </span>

<a id='item1'></a>

## 1. Create and Explore Dataset

### Imports and Initialization

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
# Import dependencies
import requests # library to handle requests
from bs4 import BeautifulSoup # website scraping librarie 

import numpy as np

import pandas as pd

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't geopy installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't folium installed
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Now assign the link of the website through which we are going to scrape the data and assign it to variable named website_url, and create a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.

In [2]:
# getting data from aspecific source (wikipedia) 
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#Find the required table are
soup = BeautifulSoup(website_url,'lxml')

# extracting the raw table inside that webpage
table = soup.find('table', {'class': 'wikitable sortable'})

### Data processing 1 :  Extract the table to a base dataframe (from Part 1)
The base dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [3]:
table_pr = table.find_all('tr')

# Extract the table header (Extract three columns PostalCode, Borough, and Neighborhood)
table_header_pr = table_pr[0].findAll('th')
table_header = [str(th).replace('<th>', '').replace('</th>', '').replace('\n', '') for th in table_header_pr]

# Extract the table body
table_body_pr = table_pr[1:]
table_body = [list(map(lambda td: str(td if not td.findAll("a") 
                                         else td.find("a").getText()).replace('<td>', '')
                          .replace('</td>', '').replace('\n', '')
                          , tr.findAll('td'))) for tr in table_body_pr]


# Create a base dataframe to clean and transform
df_base = pd.DataFrame(table_body, columns=table_header)

### Data processing 2 :  Ignore cells with a borough that is not assigned (from Part 1)

In [4]:
# replace "Not assigned" to NaN : To use dropna function for dropping the cells that have an Not assigned borough
df_base.replace("Not assigned", np.nan, inplace=True)

# Drop cells with "Not assigned" borough 
df_base.dropna(subset=["Borough"], inplace=True)

### Data processing 3 :  Replace Not assigned neighborhood by the borough (from Part 1)
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [5]:
df_base.Neighbourhood.fillna(df_base.Borough, inplace=True)

### Data processing 4 :  Combine neighborhood that have the same postal code into one row (from Part 1)
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [6]:
# Rename Postcode column to PostalCode to be used in the final Dataframe
df_base.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

# Combine neighboorhood Strings that have the same postal code to one row 
df_toronto_part1 = df_base.groupby(['PostalCode', "Borough"])[["Neighbourhood"]].agg(lambda x: ', '.join(x)).reset_index()

###  Data processing 4 -  Extracting Latitude and Longtitude for each Postal code in Toronto (from Part 2)

Using the csv file proposed in the course to create the dataframe with the geographical coordinates for each postal code

In [7]:
!wget -q -O "geographical_coordinates.csv" http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [8]:
df_coordinates = pd.read_csv('geographical_coordinates.csv')
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [9]:
# merge the datafarame of toronto neighborhoods with toronto postal code coordinates(Latitude and Longtitude)
df_toronto = df_toronto_part1.join(df_coordinates.set_index('PostalCode'), on='PostalCode')

#### Visualize the appropriate pandas Dataframe - with Latitude & Longitude

In [10]:
# Visualizing the result Dataframe
df_toronto.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#####  <span style="color:red">Information :</span> The Dataframe is displayed in an ordered form 

### Create a map of Toronto with neighborhoods superimposed on top

##### Use geopy library to get the latitude and longitude values of Toronto.

In [11]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(toronto_latitude, toronto_longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


##### Create a map of Toronto's neighborhoods 

In [12]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.4)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

### Create Map to display the neighbourhoods of the boroughs that contain the word Toronto
##### Create the dataframe of the boroughs that contain the word Toronto

In [13]:
toronto_boroughs_data = df_toronto[df_toronto['Borough'].str.contains('Toronto')].reset_index()
toronto_boroughs_data.head(10)

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,47,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


#### Here we display the boroughs concerned by the exploration

In [14]:
print([toronto_borough for toronto_borough in toronto_boroughs_data['Borough'].unique()])

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']


##### Create the Map to display the neighbourhoods of the boroughs that contain the word Toronto

In [15]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
map_toronto_boroughs = folium.Map(location = [toronto_latitude + 0.01, toronto_longitude], zoom_start = 12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_boroughs_data['Latitude'], toronto_boroughs_data['Longitude'], 
                                           toronto_boroughs_data['Borough'], toronto_boroughs_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.7).add_to(map_toronto_boroughs)

map_toronto_boroughs

#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [16]:
toronto_boroughs_data.loc[0, 'Neighbourhood']

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [17]:
neighborhood_latitude = toronto_boroughs_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_boroughs_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_boroughs_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venues that are in "The Beaches" within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [18]:
CLIENT_ID = 'HR4QZB43EERBJN3AGW3SSB5QQYPFICMLH0TDR1HA5QMW0UUR' # your Foursquare ID
CLIENT_SECRET = 'CW25314IKRJFWAJZD2VXQ5FIJQPYXPHBEP5NB1TWHKK32WMV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HR4QZB43EERBJN3AGW3SSB5QQYPFICMLH0TDR1HA5QMW0UUR
CLIENT_SECRET:CW25314IKRJFWAJZD2VXQ5FIJQPYXPHBEP5NB1TWHKK32WMV


Send the GET request and examine the resutls

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c013bc54c1f676746b5647d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b8daea1f964a520480833e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d11b941735',
         'name': 'Pub',
         'pluralName': 'Pubs',
         'primary': True,
         'shortName': 'Pub'}],
       'id': '4b8daea1f964a520480833e3',
       'location': {'address': '676 Kingston Rd.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Main St.',
        'distance': 460,
        'formattedAddress': ['676 Kingston Rd. (at Main St.)',
         'Toronto ON M4E 1R4',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

Use **get_category_type** function to get the venue category name

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Grover Pub and Grub,Pub,43.679181,-79.297215
1,Starbucks,Coffee Shop,43.678798,-79.298045
2,Domino's Pizza,Pizza Place,43.679058,-79.297382
3,Upper Beaches,Neighborhood,43.680563,-79.292869


And how many venues were returned by Foursquare from a specific neighborhood ?

In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in the boroughs that contain the word Toronto
### The boroughs for exploration are : 'East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto'

##### Let's create a function to repeat the same process to all the neighborhoods in : 'East Toronto', 'Central Toronto', 'Downtown Toronto', and 'West Toronto'

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_boroughs_venues*.

In [24]:
# type your answer here    
toronto_boroughs_venues = getNearbyVenues(toronto_boroughs_data['Neighbourhood'], toronto_boroughs_data['Latitude'], 
                                   toronto_boroughs_data['Longitude'])

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

#### Let's check the size of the resulting dataframe

In [25]:
print(toronto_boroughs_venues.shape)
toronto_boroughs_venues.head()

(1701, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Let's check how many venues were returned for each neighborhood

In [26]:
pd.DataFrame(toronto_boroughs_venues.groupby('Neighborhood').size(), columns=["Venues Count"])

,Venues Count
Neighborhood,
"Adelaide, King, Richmond",100
Berczy Park,55
"Brockton, Exhibition Place, Parkdale Village",22
Business reply mail Processing Centre969 Eastern,18
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",13
"Cabbagetown, St. James Town",46
Central Bay Street,86
"Chinatown, Grange Park, Kensington Market",100
Christie,16


#### Let's find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(toronto_boroughs_venues['Venue Category'].unique())))

There are 232 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [28]:
# one hot encoding
toronto_boroughs_onehot = pd.get_dummies(toronto_boroughs_venues[['Venue Category']], prefix="", prefix_sep="")
# toronto_boroughs_onehot2 = pd.concat([toronto_boroughs_venues['Neighborhood'], toronto_boroughs_onehot], axis=1)

# add neighborhood column back to dataframe
toronto_boroughs_onehot['Neighbourhood'] = toronto_boroughs_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_boroughs_onehot.columns[-1]] + list(toronto_boroughs_onehot.columns[:-1])
toronto_boroughs_onehot = toronto_boroughs_onehot[fixed_columns]

# Display result
toronto_boroughs_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [29]:
toronto_boroughs_onehot.shape

(1701, 233)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
toronto_boroughs_grouped = toronto_boroughs_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_boroughs_grouped.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,...,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.000000
1,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
3,Business reply mail Processing Centre969 Eastern,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.055556
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,0.00,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000


#### Let's confirm the new size

In [31]:
toronto_boroughs_grouped.shape

(38, 233)

#### Let's print each neighborhood along with the top 5 most common venues

In [32]:
num_top_venues = 5

for hood in toronto_boroughs_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_boroughs_grouped[toronto_boroughs_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1    Restaurant  0.05
2  Cocktail Bar  0.05
3    Steakhouse  0.04
4   Cheese Shop  0.04


----Brockton, Exhibition Place, Parkdale Village----
                   venue  freq
0            Coffee Shop  0.14
1         Breakfast Spot  0.09
2  Performing Arts Venue  0.09
3                   Café  0.09
4          Grocery Store  0.05


----Business reply mail Processing Centre969 Eastern----
         venue  freq
0  Yoga Studio  0.06
1   Restaurant  0.06
2   Smoke Shop  0.06
3          Spa  0.06
4   Skate Park  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0      Airport Lounge  0.15
1     Airport Service  0.15
2    Airport

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_boroughs_grouped['Neighbourhood']

for ind in np.arange(toronto_boroughs_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_boroughs_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Hotel,Restaurant,Bar,Clothing Store,Gym
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Café,Cheese Shop,Pub
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Performing Arts Venue,Bar,Caribbean Restaurant,Stadium,Italian Restaurant,Furniture / Home Store,Burrito Place
3,Business reply mail Processing Centre969 Eastern,Yoga Studio,Auto Workshop,Park,Pizza Place,Recording Studio,Butcher,Restaurant,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Boutique,Sculpture Garden,Boat or Ferry,Airport Gate,Airport Food Court,Airport,Harbor / Marina


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 4 clusters.

In [35]:
# set number of clusters
kclusters = 4

toronto_boroughs_grouped_clustering = toronto_boroughs_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_boroughs_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       3, 0, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [36]:
toronto_boroughs_merged = toronto_boroughs_data

# add clustering labels
toronto_boroughs_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_boroughs_merged = toronto_boroughs_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_boroughs_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Pizza Place,Coffee Shop,Pub,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Fruit & Vegetable Store,Brewery,Bubble Tea Shop,Restaurant
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Gym,Italian Restaurant,Sushi Restaurant,Burrito Place,Pub,Ice Cream Shop,Burger Joint,Movie Theater,Sandwich Place
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Coworking Space,Seafood Restaurant,Sandwich Place
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


Finally, let's visualize the resulting clusters

In [39]:
# create map
map_clusters = folium.Map(location=[toronto_latitude + 0.01, toronto_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i*0.8) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_boroughs_merged['Latitude'], toronto_boroughs_merged['Longitude'], toronto_boroughs_merged['Neighbourhood'], toronto_boroughs_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [40]:
toronto_boroughs_merged.loc[toronto_boroughs_merged['Cluster Labels'] == 0, toronto_boroughs_merged.columns[[1] + list(range(5, toronto_boroughs_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,-79.293031,0,Neighborhood,Pizza Place,Coffee Shop,Pub,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Fruit & Vegetable Store,Brewery,Bubble Tea Shop,Restaurant
2,M4L,-79.315572,0,Park,Gym,Italian Restaurant,Sushi Restaurant,Burrito Place,Pub,Ice Cream Shop,Burger Joint,Movie Theater,Sandwich Place
3,M4M,-79.340923,0,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Coworking Space,Seafood Restaurant,Sandwich Place
4,M4N,-79.388790,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,M4P,-79.390197,0,Park,Sandwich Place,Grocery Store,Breakfast Spot,Hotel,Burger Joint,Clothing Store,Food & Drink Shop,Yoga Studio,Dumpling Restaurant
6,M4R,-79.405678,0,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Fast Food Restaurant,Spa,Mexican Restaurant,Sandwich Place,Salon / Barbershop,Diner
7,M4S,-79.388790,0,Sandwich Place,Dessert Shop,Seafood Restaurant,Pharmacy,Sushi Restaurant,Pizza Place,Coffee Shop,Italian Restaurant,Café,Indoor Play Area
8,M4T,-79.383160,0,Intersection,Gym,Park,Playground,Trail,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant
9,M4V,-79.400049,0,Coffee Shop,Pub,Supermarket,Light Rail Station,Convenience Store,Sushi Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Pizza Place,Bagel Shop


#### Cluster 2

In [41]:
toronto_boroughs_merged.loc[toronto_boroughs_merged['Cluster Labels'] == 1, toronto_boroughs_merged.columns[[1] + list(range(5, toronto_boroughs_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,M5W,-79.374846,1,Coffee Shop,Restaurant,Café,Pub,Cocktail Bar,Seafood Restaurant,Hotel,Cosmetics Shop,Bakery,Creperie


#### Cluster 3

In [42]:
toronto_boroughs_merged.loc[toronto_boroughs_merged['Cluster Labels'] == 2, toronto_boroughs_merged.columns[[1] + list(range(5, toronto_boroughs_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M5G,-79.387383,2,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bar,Ice Cream Shop,Bubble Tea Shop,Chinese Restaurant,Burger Joint,Indian Restaurant
24,M5R,-79.405678,2,Café,Coffee Shop,Sandwich Place,Pizza Place,Cosmetics Shop,History Museum,French Restaurant,Vegetarian / Vegan Restaurant,Pharmacy,American Restaurant
27,M5V,-79.394420,2,Airport Lounge,Airport Terminal,Airport Service,Boutique,Sculpture Garden,Boat or Ferry,Airport Gate,Airport Food Court,Airport,Harbor / Marina


#### Cluster 4

In [43]:
toronto_boroughs_merged.loc[toronto_boroughs_merged['Cluster Labels'] == 3, toronto_boroughs_merged.columns[[1] + list(range(5, toronto_boroughs_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M5N,-79.416936,3,Garden,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
